In [85]:
import numpy as np
import matplotlib.pyplot as plt

n = 2 # Number of unit cells in each direction
a = 1.0e-10 # Lattice constant

lambda_X = 1.54e-10 # X-ray wavelength in Angstroms

f_O = 8 # Atomic form factor for Oxygen
f_Cu = 29 # Atomic form factor for Copper

# Diccionario para almacenar los factores de forma atómicos:
f = {
    "O": f_O,
    "Cu": f_Cu
}

# Esto al principio lo tenia así, pero le pregunte chatgpt y me dijo que para tenerlos en el mismo array era mejor separarlos usando un diccionario. Le pregunte por que me salío un error
# y lo tenía originalmente así:
# base_O_Cu = np.array([[0, 0, 0], [1/2, 1/2, 1/2]],[[1/4, 1/4, 1/4], [1/4, 3/4, 3/4], [3/4, 1/4, 3/4], [3/4, 3/4, 1/4]], dtype=float) # BCC Oxygen, FCC Copper
# Pero ahora aprendi a usar diccionarios
base_O = a * np.array([
    [0,   0,   0  ],
    [1/2, 1/2, 1/2]
], dtype=float)  # BCC (2 átomos)

base_Cu = a * np.array([
    [1/4, 1/4, 1/4],
    [1/4, 3/4, 3/4],
    [3/4, 1/4, 3/4],
    [3/4, 3/4, 1/4]
], dtype=float)  # FCC (4 átomos)

# Diccionario para almacenar las bases atómicas:
base = {
    "O": base_O,
    "Cu": base_Cu
}

lattice_vectors = a * np.array([[1, 0, 0],[0, 1, 0],[0, 0, 1]], dtype=float) # Cubic cell

Miller_indexes = np.stack(np.mgrid[0:n+1, 0:n+1, 0:n+1], axis=-1).astype(float)  # (n+1, n+1, n+1, 3) Miller indices grid

# Miller_indexes[0,0,0] = np.nan, np.nan, np.nan # Set (0,0,0) to NaN for clarity

# Calculos para obtener el factor de Estructura

In [86]:
# Calculate reciprocal lattice vectors
reciprocal_vectors = np.zeros((3, 3), dtype=float) # To store reciprocal lattice vectors
for i in range(3):
    reciprocal_vectors[i] = 2 * np.pi * np.cross(lattice_vectors[np.mod(i+1, 3)], lattice_vectors[np.mod(i+2, 3)])  / np.linalg.det(lattice_vectors)

# Calculate reciprocal lattice vector
G = Miller_indexes @ reciprocal_vectors # (n+1, n+1, n+1, 3) Reciprocal lattice vectors grid

# Calculate phase factors
phase_O = np.tensordot(G, base["O"].T, axes=([-1],[0]))  # (n+1, n+1, n+1, 2) Phase factors for Oxygen
phase_Cu = np.tensordot(G, base["Cu"].T, axes=([-1],[0]))  # (n+1, n+1, n+1, 4) Phase factors for Copper

# Function to calculate structure factor
S_G = f["O"] * np.sum(np.exp(- 1j * phase_O), axis=-1) + f["Cu"] * np.sum(np.exp(- 1j * phase_Cu), axis=-1)  # (n+1, n+1, n+1) Structure factor

I_G = np.abs(S_G)**2  # (n+1, n+1, n+1) Intensity

# Calculo de distancia interplanar d y angulo

In [ ]:
d = 2 * np.pi / np.linalg.norm(G, axis=-1)  # (n+1, n+1, n+1) Interplanar spacing

theta = np.arcsin(lambda_X / (2 * d))  # (n+1, n+1, n+1) Bragg angle in radians

theta = np.degrees(theta).reshape(-1,1)  # Convert to degrees y hacer lista de escalares

# Ordenar por angulo
order = np.argsort(theta) # Indices de ordenamiento

# Ordenar los arrays
theta_ord = theta[order]
I_G_ord = I_G.reshape(-1,1)[order] # Hacer lista de escalares
Miller_indexes_ord = Miller_indexes.reshape(-1,3)[order] # Hacer lista de tuplas


C:\Users\david\AppData\Local\Temp\ipykernel_17112\4113326380.py:1: RuntimeWarning: divide by zero encountered in divide
  d = 2 * np.pi / np.linalg.norm(G, axis=-1)  # (n+1, n+1, n+1) Interplanar spacing
C:\Users\david\AppData\Local\Temp\ipykernel_17112\4113326380.py:3: RuntimeWarning: invalid value encountered in arcsin
  theta = np.arcsin(lambda_X / (2 * d))  # (n+1, n+1, n+1) Bragg angle in radians


In [88]:
# Comporobar resultados
indice = 3 # Example Miller index to inspect
print("Indice de Miller: ", Miller_indexes_ord[indice], "; Intensidad:", I_G_ord[indice], "; Angulo:", theta_ord[indice]) # Example output for Miller index (1,1,1)

Indice de Miller:  [[0. 0. 0.]] ; Intensidad: [[17424.]] ; Angulo: [[0.]]
